In [2]:
%pip install pythautomata


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Creating a moore machine

#### Dependencies

In [17]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.moore_state import MooreState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy as ComparisonStrategy
from pythautomata.model_exporters.dot_exporting_mm_strategy import DotExportingMMStrategy

#### Implementation

In [7]:
# Input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

# Define states
stateA = MooreState("State A", output_alphabet['0'])
stateB = MooreState("State B", output_alphabet['1'])
stateC = MooreState("State C", output_alphabet['1'])

# Add transitions
stateA.add_transition(a, stateA)
stateA.add_transition(b, stateB)
stateB.add_transition(a, stateC)
stateB.add_transition(c, stateB)

# Hole state
hole_state = MooreState(name="hole", value=SymbolStr('0'))

moore_machine = MooreMachineAutomaton(input_alphabet, output_alphabet, stateA, set([stateA, stateB, stateC]), ComparisonStrategy, "moore machine with 3 states", [DotExportingMMStrategy])

# Moore machine is generated in the directory ./output_models/moore_machines/
moore_machine.export('./output_models/moore_machines/')

Unexpected exception when exporting moore machine with 3 states: <class 'AttributeError'>


### Moore machine generator

#### Dependencies

In [18]:
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine

#### Implementation

In [9]:
# Amount of states
size = 20

generated_moore_machine = generate_moore_machine(input_alphabet, output_alphabet, size)

# Moore machine is generated in the directory ./output_models/moore_machines/
generated_moore_machine.export('./output_models/moore_machines/')

Unexpected exception when exporting Moore Machine - 0f982dfc1d424b3cb1050837ab3f9a0d: <class 'graphviz.backend.execute.ExecutableNotFound'>


### DFA - moore machine comparator and converter

#### Dependencies

In [14]:
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines

#### Implementation

In [15]:
# Tomitas DFA example
tomitas = TomitasGrammars.get_automaton_1()

# Convert DFA to moore machine
converted_moore_machine = AutomataConverter.convert_dfa_to_moore_machine(tomitas)

# Tomitas moore machine
tomitas_moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Comparate automatons
converted_moore_machine.__eq__(tomitas_moore_machine)

True

In [16]:
%pip install pymodelextractor

     ---------------------------------------- 69.4/69.4 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 78.8/78.8 kB 4.3 MB/s eta 0:00:00

  Attempting uninstall: pythautomata
    Found existing installation: pythautomata 0.21.0
    Uninstalling pythautomata-0.21.0:
      Successfully uninstalled pythautomata-0.21.0



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Moore machine L*

#### Dependencies

In [ ]:
from pymodelextractor.learners.observation_table_learners.lstar_learner import LStarLearner
from pymodelextractor.teachers.automaton_teacher import Teacher


#### Implementation